In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip
!wget http://www.gatsby.ucl.ac.uk/~chuwei/data/EachMovie/Vote.zip
!unzip Vote.zip

--2022-10-11 16:43:09--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  28.7MB/s    in 0.2s    

2022-10-11 16:43:10 (28.7 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
--2022-10-11 16:43:10--  http://www.gatsby.ucl.ac.uk/~chuwei/data/EachMovie/Vote.zip
Resolving www.gatsby.ucl.ac.uk (www.gatsby.ucl.ac.uk)... 128.40.213.242
Connecting to www.gatsby.ucl.ac.uk (www.gatsby.ucl.ac.uk)|128.40.213.242|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [ ]:
!ls

ml-1m  ml-1m.zip  sample_data  Vote.txt  Vote.zip


In [ ]:
!pip install  recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 339 kB 38.5 MB/s 
     |████████████████████████████████| 310 kB 70.9 MB/s 
     |████████████████████████████████| 636 kB 74.0 MB/s 
     |████████████████████████████████| 120 kB 75.2 MB/s 
     |████████████████████████████████| 5.3 MB 51.6 MB/s 
     |████████████████████████████████| 355 kB 73.1 MB/s 
     |████████████████████████████████| 12.4 MB 50.0 MB/s 
     |████████████████████████████████| 61 kB 296 kB/s 
     |████████████████████████████████| 11.8 MB 59.2 MB/s 
     |████████████████████████████████| 395 kB 67.0 MB/s 
     |████████████████████████████████| 163 kB 63.5 MB/s 
     |████████████████████████████████| 7.6 MB 58.8 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=697445 sha256=f399c34a552bc7e9b20a8221898784bd21e4cda023ed1d3b6d3e181027d5d973
  Stored in directory: /root/.cache/pip/wh

In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from surprise import (
    Dataset,
    SVD,
    KNNBasic,
    accuracy,
    Reader,
    BaselineOnly
    
)
import pandas as pd
from surprise.model_selection import cross_validate, LeaveOneOut,train_test_split
from recommenders.evaluation.python_evaluation import ndcg_at_k
from sklearn.metrics import ndcg_score
import numpy as np


In [ ]:
def Make_test1(data):
    LOOCV = LeaveOneOut(n_splits=1, random_state=1)

    # split trainset, lefouts set
    for trainSet, leftOuts in LOOCV.split(data):
        # Make full test set
        bigTestSet = trainSet.build_anti_testset()

    # Make small testset 1M rows from biTestSet 
    big_testset_df = pd.DataFrame(bigTestSet, columns = ['user_id', 'movie_id', 'rating'])
    small_testset_df = big_testset_df.sample(1000000)

    # Add leftouts set to small testset to make sure there are lefouts in testset
    # This is better for calculate hit rate later.
    small_testset_df = pd.concat([small_testset_df, pd.DataFrame(leftOuts, columns = ['user_id', 'movie_id', 'rating'])])

    # remove duplicate in testset
    small_testset_df.drop_duplicates(subset = ['user_id', 'movie_id'], inplace = True)

    # convert to list to fit model
    testList = small_testset_df.values.tolist()
    return [testList, trainSet, leftOuts]

In [ ]:
def Make_test2(data):
    # Make trainset and leftouts set.
    LOOCV = LeaveOneOut(n_splits=1, random_state=1)
    for trainSet, leftOuts in LOOCV.split(data):
        pass

    # GENERAL IDEA:
    # Make DF from trainset.
    # For each movie
    # Generate all user did not rate that movie
    # Get 1000 sample for each movie
    # Testset is a join all the sample and leftouts set
    
    # make DataFrame testset
    trainset_df = pd.DataFrame(trainSet.build_testset(), columns = ['user_id', 'item_id', 'rating'])

    # Create test_list to contains list of testsets
    test_list = []

    # For each movie in dataset
    for i in range(1,1649):
        try:
            # Get all the ratings of moive in testset
            user_id_single_df = trainset_df[trainset_df.item_id == i]
            user_id_single_list = user_id_single_df.user_id.tolist()

            # Generate all user
            all_user_id_list = [*range(1,74424+1)]

            # Add all user to the all user to make duplicate
            final_user_id_list = all_user_id_list + user_id_single_list

            # Create item_id list 
            item_ls = [i]*len(final_user_id_list)

            # Create DF of the movie id, all the user_id (have duplicates)
            # and the ratings is random.
            tempdf = pd.DataFrame({'user_id': final_user_id_list, 'item_id':item_ls, 'rating':item_ls})

            # Remove all the duplicated user_id (if there is more than one in DF, remove them all)
            # Ater this step the DF only have the ratings that is not in the trainset.
            nodup = tempdf.drop_duplicates(subset='user_id', keep = False)

            # Add random sample 1000 rows DF to item_list
            test_list.append(nodup.sample(1000))
        except:
            print('erro: ',i)
            pass

    # Add LeftOuts set into test_list to make sure all the leftouts is in testset.
    # for later Hit Rate calculation.
    test_list.append(pd.DataFrame(leftOuts, columns = ['user_id', 'item_id', 'rating']))

    # Merge all the DF in test_list
    test_df = pd.concat(test_list, ignore_index=True)


    # Drop duplicates in DF (only keep 1 ratings of user to item in DF)
    test_df.drop_duplicates(subset = ['user_id', 'item_id'], keep = 'last')

    # Conver to list to fit model.
    test_list1 = test_df.values.tolist()

    return [test_list1, trainSet, leftOuts]    

In [ ]:
def HitRate(svd, data, k, Make_test = Make_test1, minimumRating=4.0):

    # Get TestList, TrainSet, Leftouts set from function Make_test.
    testList, trainSet, leftOuts = Make_test(data)

    # Fit model with trainset.
    svd.fit(trainSet)

    # Make predict with testset
    Predictions1 = svd.test(testList)

    # Make DF of predictions
    prediction_df = pd.DataFrame(Predictions1, columns = ['user_id', 'item_id', 'rating', 'prediction', 'detail'])

    # Sort ratings values descending for each user_id
    prediction_df.sort_values(['user_id', 'prediction'], ascending=False, inplace=True, ignore_index=True)

    # Remove all the ratings smaller than minimumRating
    prediction_df = prediction_df[prediction_df.prediction >= minimumRating]

    # Make a new user_id columns for groupby
    # When groupby the user_id columns will become index
    # After use head() function, the index is the index of DF before groupby
    # As a result we will lose the user_id
    # Make a new columns only to groupby on that columns
    # The user_id columns will be preserve
    prediction_df['u_id'] = prediction_df['user_id']

    # Groupby and get top k ratings of the user.
    top10predictions = prediction_df.groupby('u_id')['user_id','item_id', 'prediction'].head(k)


    # Calculate hit rate.
    hits = 0

    # for each leftout in leftouts set.
    for leftOut in leftOuts:

        # Get user_id and movie_id of that leftout
        user_id = leftOut[0]
        movie_id = leftOut[1]

        # Get the DF of that user in DF of top 10 predictions of every users.
        top10 = top10predictions[top10predictions.user_id == user_id]

        # If the DF has length
        if len(top10[top10.item_id == movie_id]):

            # Add one to hits.
            hits += 1

    # return hit rate.
    return hits/len(leftOuts)

In [ ]:
def NDCG(predictions):
    # Make DF from predictoins
    df = pd.DataFrame(predictions, columns = ['user_id', 'item_id', 'rating', 'prediction', 'detail'])

    # Make array of ratings and predictions.
    rating = np.asarray([df.rating])
    prediction = np.asarray([df.prediction])

    # Calculate Ndcg 
    ndcg_sci = ndcg_score(rating, prediction)
    return ndcg_sci

In [ ]:
def SVD_test(name, data, test_size = 0.3, 
             n_factors=100, n_epochs=20, biased=True,
             lr_all=0.005, reg_all=0.02,
             Make_test = Make_test1,
             init_mean = 0, init_std_dev = 0.1):  

    # Split trainset, testset from data
    trainset, testset = train_test_split(data, test_size=test_size)

    # Create SVD model
    svd = SVD(n_factors = n_factors, n_epochs=n_epochs, biased=biased, 
              init_mean = init_mean, init_std_dev = init_std_dev,
              lr_all=lr_all, reg_all=reg_all)
    
    # Fit SVD model and make predictions
    svd.fit(trainset)
    predictions = svd.test(testset)

    # Calculate measures.
    rmse = accuracy.rmse(predictions, verbose = False)
    hitRate10 = HitRate(svd, data, 10, Make_test = Make_test)
    hitRate3 = HitRate(svd, data, 3, Make_test = Make_test)
    ndcg = NDCG(predictions)
    
    ls = [name, rmse, hitRate10, hitRate3, ndcg]
    return pd.DataFrame([ls], columns = ['Name', 'RSME', 'Hit_Rate(k=10)', 'Hit_Rate(k=3)', 'NDCG'])

In [ ]:
# Read file ML_1M
file_path = 'ml-1m/ratings.dat'
reader = Reader(line_format="user item rating timestamp", sep="::")
movieLen = Dataset.load_from_file(file_path, reader=reader)

In [ ]:
# Read file EachMovie


file_path2 = 'Vote.txt'

# Read file to DF
eachMovie_df = pd.read_csv('Vote.txt', sep = '\t', header=None)

# Only get user_id, item_id, and rating
eachMovie_df = eachMovie_df[eachMovie_df.columns[0:-2]]
eachMovie_df.columns = ['user_id', 'item_id', 'rating']

# The rating range is 0-1, make it to range 0-5
eachMovie_df['rating'] = eachMovie_df.rating*5

# Conver DF to surprise data
reader2 = Reader(rating_scale=(0, 5))
eachMovie = Dataset.load_from_df(eachMovie_df, reader=reader2)

In [ ]:
SVD_test('normal', movieLen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Name,RSME,Hit_Rate(k=10),Hit_Rate(k=3),NDCG
0,normal,0.882643,0.206291,0.103146,0.992347


In [ ]:
SVD_test('biased = False', movieLen, biased = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Name,RSME,Hit_Rate(k=10),Hit_Rate(k=3),NDCG
0,biased = False,0.883563,0.227318,0.12202,0.99238


In [ ]:
SVD_test('normal', eachMovie, Make_test = Make_test2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Name,RSME,Hit_Rate(k=10),Hit_Rate(k=3),NDCG
0,normal,1.158625,0.194809,0.171909,0.989488


In [ ]:
SVD_test('biased = False', eachMovie, Make_test = Make_test2, biased = False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,Name,RSME,Hit_Rate(k=10),Hit_Rate(k=3),NDCG
0,biased = False,1.202763,0.159912,0.147882,0.98829
